In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import chardet

# Deteksi encoding file
with open('data2.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # baca 10KB pertama
    print(f"Detected encoding: {result['encoding']}")

# Baca dengan encoding yang terdeteksi
raw_data = pd.read_csv('data2.csv', encoding=result['encoding'])
raw_data.head()

Detected encoding: ISO-8859-1


,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [3]:
data_clean = raw_data[['Tweet', 'HS']].copy() 
data_clean.columns = ['text', 'label']
data_clean.head()

,text,label
0,- disaat semua cowok berusaha melacak perhatia...,1
1,RT USER: USER siapa yang telat ngasih tau elu?...,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1


In [4]:
# Cek sebelum cleaning - perbaiki pattern
print("Sebelum cleaning:")
print(data_clean['text'].str.contains(r'\bUSER\b', regex=True).sum())

# hapus kata "USER" (tanpa @)
data_clean['text'] = data_clean['text'].str.replace(r'\bUSER\b', '', regex=True)

data_clean['text'] = data_clean['text'].str.replace(r'^RT USER:\s*', '', regex=True)

# Cek setelah cleaning
print("Setelah cleaning:")
print(data_clean['text'].str.contains(r'\bUSER\b', regex=True).sum())

data_clean.head()

Sebelum cleaning:
7680
Setelah cleaning:
0


,text,label
0,- disaat semua cowok berusaha melacak perhatia...,1
1,RT : siapa yang telat ngasih tau elu?edan sar...,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0
3,AKU ITU AKU\n\nKU TAU MATAMU SIPIT TAPI DILI...,0
4,Kaum cebong kapir udah keliatan dongoknya da...,1


In [5]:
# Atau jika ingin lebih spesifik (hashtag dengan huruf/angka/underscore)
data_clean['text'] = data_clean['text'].str.replace(r'#[A-Za-z0-9_]+', '', regex=True)

# Hapus karakter unicode bermasalah (emoji yang rusak)
data_clean['text'] = data_clean['text'].str.replace(r'\\x[0-9a-fA-F]{2}', '', regex=True)

# Hapus \n yang masih tersisa
data_clean['text'] = data_clean['text'].str.replace(r'\\n', ' ', regex=True)

data_clean['text'] = data_clean['text'].str.replace(r'^RT :\s*', '', regex=True)
data_clean['text'] = data_clean['text'].str.replace(r'^-\s*', '', regex=True)
# Hapus \n (newline)
data_clean['text'] = data_clean['text'].str.replace(r'\n', ' ', regex=True)

# Hapus karakter non-ASCII yang bermasalah
data_clean['text'] = data_clean['text'].str.encode('ascii', errors='ignore').str.decode('ascii')

#hapus karakter yang tidak diinginkan
data_clean['text'] = data_clean['text'].str.replace(r'[^\w\s,.!?\'\"-]', '', regex=True)

# hapus link
data_clean['text'] = data_clean['text'].str.replace(r'http\S+|www\S+|https\S+', '', regex=True)
# Hapus mention (username Twitter)
data_clean['text'] = data_clean['text'].str.replace(r'@\w+', '', regex=True)

# Hapus angka di depan text
data_clean['text'] = data_clean['text'].str.replace(r'^\d+\.?\s*', '', regex=True)

# Bersihkan whitespace berlebih
data_clean['text'] = data_clean['text'].str.replace(r'\s+', ' ', regex=True)

# Hapus spasi di awal dan akhir
data_clean['text'] = data_clean['text'].str.strip()

data_clean.head(50)

,text,label
0,disaat semua cowok berusaha melacak perhatian ...,1
1,siapa yang telat ngasih tau elu?edan sarap gue...,0
2,"Kadang aku berfikir, kenapa aku tetap percaya ...",0
3,AKU ITU AKU KU TAU MATAMU SIPIT TAPI DILIAT DA...,0
4,Kaum cebong kapir udah keliatan dongoknya dari...,1
5,Ya bani taplak dkk ',1
6,deklarasi pilkada 2018 aman dan anti hoax warg...,0
7,Gue baru aja kelar re-watch Aldnoah Zero!!! pa...,0
8,Nah admin belanja satu lagi port terbaik nak m...,0
9,Enak lg klo smbil ngewe',0


In [6]:
# Hapus koma double atau lebih
data_clean['text'] = data_clean['text'].str.replace(r',{2,}', ',', regex=True)

# Atau untuk menghapus semua koma berlebih
data_clean['text'] = data_clean['text'].str.replace(r',+', ',', regex=True)

# Bersihkan whitespace berlebih
data_clean['text'] = data_clean['text'].str.replace(r'\s+', ' ', regex=True)

# Hapus spasi di awal dan akhir
data_clean['text'] = data_clean['text'].str.strip()

In [7]:
data_clean.head(50)

,text,label
0,disaat semua cowok berusaha melacak perhatian ...,1
1,siapa yang telat ngasih tau elu?edan sarap gue...,0
2,"Kadang aku berfikir, kenapa aku tetap percaya ...",0
3,AKU ITU AKU KU TAU MATAMU SIPIT TAPI DILIAT DA...,0
4,Kaum cebong kapir udah keliatan dongoknya dari...,1
5,Ya bani taplak dkk ',1
6,deklarasi pilkada 2018 aman dan anti hoax warg...,0
7,Gue baru aja kelar re-watch Aldnoah Zero!!! pa...,0
8,Nah admin belanja satu lagi port terbaik nak m...,0
9,Enak lg klo smbil ngewe',0


In [8]:
# ubah 1 menjadi hate, 0 menjadi neutral
data_clean['label'] = data_clean['label'].replace({1: 'hate', 0: 'neutral'})

In [12]:
data_clean['text'] = data_clean['text'].astype(str)
data_clean.to_csv('cleaned_2.csv', index=False)